Ten notatnik zawiera kod dostrajania Soft-Prompt przy użyciu modelu językowego GPT-2 (z głowicą do generowania tekstu).
Na podstawie: https://gist.github.com/mf1024/3df214d2f17f3dcc56450ddf0d5a4cd7

#Importy

Installing and importing necessary libraries.

In [ ]:
# Install transformers library.
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.3 MB/s eta 0:00:00


In [ ]:
import io
import os
import torch
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
#from ml_things import plot_dict, plot_confusion_matrix, fix_text
from sklearn.metrics import classification_report, accuracy_score
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2Config,
                          GPT2Tokenizer,
                          AdamW,
                          get_linear_schedule_with_warmup,
                          GPT2ForSequenceClassification,
                          GPT2LMHeadModel,
                          GPT2Model)

In [ ]:
import torch.nn as nn

#Weights&Biases

Instalacja i import Weights&Biases

In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.8/218.8 kB 25.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.1 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=b3963972ed6525fc7f8fdf8a004a798dfb7bc541a2235b2fee629bbc3b2d6fc5
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
import wandb

In [ ]:
wandb.login()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import os

#Ścieżki do danych

Wskazanie ścieżek danych.

In [ ]:
TRAIN_DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/PW PROJEKT/csvs/dets/raw_data/output"

In [ ]:
TRAIN_DATA_FILE = "train_data.csv"

In [ ]:
VAL_DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/PW PROJEKT/csvs/dets/raw_data/output"

In [ ]:
VAL_DATA_FILE = "val_data.csv"

In [ ]:
SAMPLE_DATA_FILE = "train_data_sample.csv"

#Urządzenie

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [ ]:
device

'cuda'

#Parametry treningu

In [ ]:
BATCH_SIZE = 1
EPOCHS = 3
LEARNING_RATE = 1e-3
WARMUP_STEPS = 0
MAX_SEQ_LEN = 230

In [ ]:
soft_prompt_len = 10

#Inicjalizacja Weights&Biases

In [ ]:
run = wandb.init(
    #Projekt W&B
    project="pw-gsn-project-gpt2lmhead",
    #Śledzone parametry
    config={
        "learning_rate": LEARNING_RATE,
        "epochs": EPOCHS,
        "soft_prompt_len": soft_prompt_len,

    })

wandb: Currently logged in as: zawemi. Use `wandb login --relogin` to force relogin


#Zbiór danych

Definicja klasy Dataset (PyTorch)

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd

class QSADataset(Dataset):
    def __init__(self, qsa_data_path = TRAIN_DATA_DIR, qsa_data_file = TRAIN_DATA_FILE):
        super().__init__()

        train_qsa_data_path = os.path.join(qsa_data_path, qsa_data_file)

        self.qsa_texts = []
        self.qsa_labels = []

        qsa_df = pd.read_csv(train_qsa_data_path)

        for idx, row in qsa_df.iterrows():
          qsa_str = f'Question: {row["question"]}; Sentence: {row["description"]}; Answer:'
          qsa_label = row["binary_label"]
          #print(qsa_str)
          #print(qsa_label)
          self.qsa_texts.append(qsa_str)
          self.qsa_labels.append(qsa_label)

    def __len__(self):
        return len(self.qsa_texts)

    def __getitem__(self, item):
        return self.qsa_texts[item], self.qsa_labels[item]

In [ ]:
qsa_dataset = QSADataset(qsa_data_path = TRAIN_DATA_DIR, qsa_data_file = TRAIN_DATA_FILE)
qsa_dataloader = DataLoader(qsa_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
val_qsa_dataset = QSADataset(qsa_data_path = VAL_DATA_DIR, qsa_data_file = VAL_DATA_FILE)
val_qsa_dataloader = DataLoader(val_qsa_dataset, batch_size=BATCH_SIZE, shuffle=False)

#Model

Zaimportowanie modelu, zamrożenie parametrów modelu, stworzenie zerowego soft-promptu.

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


In [ ]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
for param in model.parameters():
  param.requires_grad = False

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
soft_prompt = torch.zeros((soft_prompt_len, 1024), requires_grad=True)

In [ ]:
soft_prompt

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True)

In [ ]:
soft_prompt.shape

torch.Size([10, 1024])

In [ ]:
optimizer = torch.optim.Adam([soft_prompt], lr=LEARNING_RATE)

#Funkcje pomocnicze

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
def validation(val_dataloader, soft_prompt):
  #wykonaj walidację, uwzględniając soft-prompt
  sum_loss = 0.0
  batch_count = 0

  predicted_labels = []
  true_labels = []

  cross_entropy_loss = nn.CrossEntropyLoss()

  for qsa in val_dataloader:
    text = tokenizer.encode(qsa[0][0])
    text_embeds = model.transformer.wte.weight[text, :] #uzyskanie embeddingów, aby można je było połączyć z soft-promptem
    with torch.no_grad():
      embeds = torch.cat((soft_prompt.to(device), text_embeds.to(device)), 0).unsqueeze(0).to(device)
      label = tokenizer.encode(qsa[1][0])
      label_index = label[0]

      outputs = model(inputs_embeds=embeds)
      softmax_logits = torch.softmax(outputs.logits, dim=-1)
      predicted_label = softmax_logits.detach().argmax(axis=-1).flatten().tolist()[-1] #ostatni token
      predicted_labels.append(predicted_label)
      loss = cross_entropy_loss(outputs.logits[:,-1,:], torch.cuda.LongTensor([label_index]))

    sum_loss += loss.clone().detach()
    batch_count += 1
    avg_loss = sum_loss/batch_count
    true_labels.append(label_index)

    accuracy = accuracy_score(true_labels, predicted_labels)

  return avg_loss, accuracy

#Trening

In [ ]:
VALID_EVERY = 3000

In [ ]:
SOFT_PROMPT_PATH = "/content/drive/MyDrive/Colab Notebooks/PW PROJEKT/csvs/dets/soft_prompt"

In [ ]:
sum_loss = 0.0
batch_count = 0

predicted_labels = []
true_labels = []

models_folder = "trained_models_lmhead"
cross_entropy_loss = nn.CrossEntropyLoss()

if not os.path.exists(models_folder):
    os.mkdir(models_folder)

for epoch in range(EPOCHS):

    print(f"EPOCH {epoch+1}/{EPOCHS} started" + '=' * 30)

    for qsa in tqdm(qsa_dataloader):
        text = tokenizer.encode(qsa[0][0])
        text_embeds = model.transformer.wte.weight[text, :]

        embeds = torch.cat((soft_prompt.to(device), text_embeds.to(device)), 0).unsqueeze(0).to(device)
        label = tokenizer.encode(qsa[1][0])
        label_index = label[0]
        outputs = model(inputs_embeds=embeds)
        softmax_logits = torch.softmax(outputs.logits, dim=-1)
        predicted_label = softmax_logits.detach().argmax(axis=-1).flatten().tolist()[-1]
        predicted_labels.append(predicted_label)

        loss = cross_entropy_loss(outputs.logits[:,-1,:], torch.cuda.LongTensor([label_index]))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        sum_loss += loss.clone().detach()
        batch_count += 1
        avg_loss = sum_loss/batch_count
        wandb.log({"Avg loss": avg_loss})
        true_labels.append(label_index)
        accuracy = accuracy_score(true_labels, predicted_labels)
        wandb.log({"Accuracy": accuracy})

        if batch_count % VALID_EVERY == 0:
          print(f"Avg loss: {sum_loss/batch_count}")
          print(f"Accuracy: {accuracy}")
          #WALIDACJA
          val_avg_loss, val_accuracy = validation(val_qsa_dataloader, soft_prompt)
          wandb.log({"Validation loss": val_avg_loss})
          wandb.log({"Validation accuracy": val_accuracy})
          print(f"Validation loss: {val_avg_loss}")
          print(f"Validation accuracy: {val_accuracy}")
          print("=" * 30)

torch.save(soft_prompt, f"{SOFT_PROMPT_PATH}/soft_prompt_{soft_prompt_len}_e{EPOCHS}_lr{LEARNING_RATE}.pt")

EPOCH 1/3 started==============================


  0%|          | 0/37981 [00:00<?, ?it/s]

Avg loss: 1.8790770769119263
Accuracy: 0.42433333333333334
Validation loss: 0.7367240190505981
Validation accuracy: 0.504
Avg loss: 1.3366159200668335
Accuracy: 0.4751666666666667
Validation loss: 0.788270890712738
Validation accuracy: 0.462
Avg loss: 1.1355249881744385
Accuracy: 0.4961111111111111
Validation loss: 0.6903929114341736
Validation accuracy: 0.557
Avg loss: 1.0334229469299316
Accuracy: 0.5091666666666667
Validation loss: 0.6916645169258118
Validation accuracy: 0.547
Avg loss: 0.9669666290283203
Accuracy: 0.518
Validation loss: 0.663616955280304
Validation accuracy: 0.602
Avg loss: 0.9207178950309753
Accuracy: 0.5278888888888889
Validation loss: 0.6566595435142517
Validation accuracy: 0.595
Avg loss: 0.8855274319648743
Accuracy: 0.5364285714285715
Validation loss: 0.654274582862854
Validation accuracy: 0.611
Avg loss: 0.8576854467391968
Accuracy: 0.5453333333333333
Validation loss: 0.6373049020767212
Validation accuracy: 0.634
Avg loss: 0.8359507322311401
Accuracy: 0.552259

  0%|          | 0/37981 [00:00<?, ?it/s]

Avg loss: 0.7708649635314941
Accuracy: 0.5837179487179487
Validation loss: 0.6090788841247559
Validation accuracy: 0.694
Avg loss: 0.7571638226509094
Accuracy: 0.5916666666666667
Validation loss: 0.5822297930717468
Validation accuracy: 0.699
Avg loss: 0.7468013167381287
Accuracy: 0.5973333333333334
Validation loss: 0.5639436841011047
Validation accuracy: 0.708
Avg loss: 0.7365172505378723
Accuracy: 0.6037708333333334
Validation loss: 0.5492985844612122
Validation accuracy: 0.707
Avg loss: 0.7269856333732605
Accuracy: 0.609686274509804
Validation loss: 0.5471891164779663
Validation accuracy: 0.726
Avg loss: 0.7190767526626587
Accuracy: 0.6147222222222222
Validation loss: 0.5451328754425049
Validation accuracy: 0.738
Avg loss: 0.7115598320960999
Accuracy: 0.6196315789473684
Validation loss: 0.5318534970283508
Validation accuracy: 0.74
Avg loss: 0.704997718334198
Accuracy: 0.62405
Validation loss: 0.5441215634346008
Validation accuracy: 0.725
Avg loss: 0.698457658290863
Accuracy: 0.628269

  0%|          | 0/37981 [00:00<?, ?it/s]

Avg loss: 0.6709775924682617
Accuracy: 0.6464871794871795
Validation loss: 0.5558394193649292
Validation accuracy: 0.717
Avg loss: 0.6662015318870544
Accuracy: 0.6496543209876543
Validation loss: 0.5112429261207581
Validation accuracy: 0.76
Avg loss: 0.6614426374435425
Accuracy: 0.6528095238095238
Validation loss: 0.49942901730537415
Validation accuracy: 0.768
Avg loss: 0.6574260592460632
Accuracy: 0.655183908045977
Validation loss: 0.49802061915397644
Validation accuracy: 0.77
Avg loss: 0.653327465057373
Accuracy: 0.6583444444444444
Validation loss: 0.4872508943080902
Validation accuracy: 0.78
Avg loss: 0.6497253775596619
Accuracy: 0.6606236559139785
Validation loss: 0.5082315802574158
Validation accuracy: 0.774
Avg loss: 0.645699143409729
Accuracy: 0.6630520833333333
Validation loss: 0.49730274081230164
Validation accuracy: 0.775
Avg loss: 0.6413031220436096
Accuracy: 0.6662424242424242
Validation loss: 0.5138555765151978
Validation accuracy: 0.757
Avg loss: 0.6375632882118225
Accura

In [ ]:
print(f"Validation loss: {val_avg_loss}")
print(f"Validation accuracy: {val_accuracy}")

Validation loss: 0.5318601727485657
Validation accuracy: 0.74


#Zapisywanie wytrenowanego soft-prompt

In [ ]:
soft_prompt

tensor([[ 0.0584,  0.0360, -0.7854,  ..., -0.4388, -0.2693, -0.6238],
        [ 0.4492, -0.4090, -0.2128,  ...,  0.5394, -0.3928, -0.5798],
        [ 0.3308, -0.1616,  0.1343,  ...,  0.2569,  0.2870, -0.4862],
        ...,
        [ 0.7763, -0.4408, -0.0224,  ...,  0.0685, -0.1097, -0.2956],
        [ 0.1662, -0.3315, -0.5440,  ..., -0.0763, -0.1549,  0.1164],
        [-0.2089, -0.2048,  0.0958,  ...,  0.3468, -0.2298,  0.2133]],
       requires_grad=True)

In [ ]:
sf_file_name = f"{SOFT_PROMPT_PATH}/soft_prompt_{soft_prompt_len}_e{EPOCHS}_lr{LEARNING_RATE}.pt"

In [ ]:
torch.save(soft_prompt, sf_file_name)

In [ ]:
sf = torch.load(sf_file_name)

In [ ]:
sf

tensor([[ 0.0584,  0.0360, -0.7854,  ..., -0.4388, -0.2693, -0.6238],
        [ 0.4492, -0.4090, -0.2128,  ...,  0.5394, -0.3928, -0.5798],
        [ 0.3308, -0.1616,  0.1343,  ...,  0.2569,  0.2870, -0.4862],
        ...,
        [ 0.7763, -0.4408, -0.0224,  ...,  0.0685, -0.1097, -0.2956],
        [ 0.1662, -0.3315, -0.5440,  ..., -0.0763, -0.1549,  0.1164],
        [-0.2089, -0.2048,  0.0958,  ...,  0.3468, -0.2298,  0.2133]],
       requires_grad=True)

In [ ]:
sf.shape

torch.Size([10, 1024])

In [ ]:
avg_loss, val_accuracy = validation(val_qsa_dataloader, sf)
print(f"Average loss = {avg_loss}")
print(f"Validation accuracy = {val_accuracy}")

Average loss = 0.48776063323020935
Validation accuracy = 0.773


#Stały prompt

Eksperyment ze stałym promptem.

In [ ]:
custom_prompt = "Does the sentence contain the answer for the question?"

In [ ]:
inputs = tokenizer.encode(custom_prompt, return_tensors="pt")
embeds = model.transformer.wte.weight[inputs, :]

In [ ]:
embeds

tensor([[[-0.2312, -0.0091, -0.0465,  ...,  0.0177, -0.0531, -0.0275],
         [ 0.0142,  0.0163, -0.0657,  ..., -0.0232, -0.0290,  0.0027],
         [ 0.1174, -0.0582,  0.1042,  ..., -0.1245, -0.0112, -0.0133],
         ...,
         [ 0.0142,  0.0163, -0.0657,  ..., -0.0232, -0.0290,  0.0027],
         [-0.0520, -0.1040, -0.0225,  ..., -0.1032, -0.0670,  0.1041],
         [-0.0013, -0.0649, -0.0011,  ..., -0.0340, -0.0065, -0.0072]]],
       device='cuda:0')

In [ ]:
embeds.squeeze().shape

torch.Size([10, 1024])

In [ ]:
avg_loss, val_accuracy = validation(val_qsa_dataloader, embeds.squeeze())
print(f"Average loss = {avg_loss}")
print(f"Validation accuracy = {val_accuracy}")

Average loss = 11.535232543945312
Validation accuracy = 0.0


#Stały few-shot prompt

Eksperyment ze stałym promptem z dwoma przykładami.

In [ ]:
few_shot_prompt = '''Does the sentence contain the answer for the question? Question: What animal is fluffy and furry?; Sentence: the cat sleeps on a windowsill; Answer:yes
  Does the sentence contain the answer for the question? Question: What can be used to cut bread?; Sentence: a person on a bike; Answer:no
  Does the sentence contain the answer for the question? '''

In [ ]:
inputs = tokenizer.encode(few_shot_prompt, return_tensors="pt")
embeds = model.transformer.wte.weight[inputs, :]

In [ ]:
avg_loss, val_accuracy = validation(val_qsa_dataloader, embeds.squeeze())
print(f"Average loss = {avg_loss}")
print(f"Validation accuracy = {val_accuracy}")

Average loss = 3.1307263374328613
Validation accuracy = 0.143


#Stały, dłuższy few-shot prompt

Eksperyment ze stałym promptem z czterema przykładami.

In [ ]:
few_shot_prompt = '''Does the sentence contain the answer for the question? Question: What animal is fluffy and furry?; Sentence: the cat sleeps on a windowsill; Answer:yes
  Does the sentence contain the answer for the question? Question: What can be used to cut bread?; Sentence: a person on a bike; Answer:no
  Does the sentence contain the answer for the question? Question: What protects skin from the sun?; Sentence: a boy playing tennis; Answer:no
  Does the sentence contain the answer for the question? Question: What people use to see better?; Sentence: cat in funny glasses; Answer:yes
  Does the sentence contain the answer for the question? '''

In [ ]:
inputs = tokenizer.encode(few_shot_prompt, return_tensors="pt")
embeds = model.transformer.wte.weight[inputs, :]

In [ ]:
avg_loss, val_accuracy = validation(val_qsa_dataloader, embeds.squeeze())
print(f"Average loss = {avg_loss}")
print(f"Validation accuracy = {val_accuracy}")

Average loss = 1.869977593421936
Validation accuracy = 0.404


#Interpretacja soft-prompt

Próba znalezienia znaczenia wytrenowanego soft-promptu poprzez znalezienie najbliższych wektorów w przestrzeni wektorów embeddingów tokenizera.

In [ ]:
sf_file_name = "/content/drive/MyDrive/Colab Notebooks/PW PROJEKT/csvs/dets/soft_prompt/soft_prompt_10_e3_lr0.001.pt"

In [ ]:
sp = torch.load(sf_file_name)

In [ ]:
sp.to(device)

tensor([[ 0.0584,  0.0360, -0.7854,  ..., -0.4388, -0.2693, -0.6238],
        [ 0.4492, -0.4090, -0.2128,  ...,  0.5394, -0.3928, -0.5798],
        [ 0.3308, -0.1616,  0.1343,  ...,  0.2569,  0.2870, -0.4862],
        ...,
        [ 0.7763, -0.4408, -0.0224,  ...,  0.0685, -0.1097, -0.2956],
        [ 0.1662, -0.3315, -0.5440,  ..., -0.0763, -0.1549,  0.1164],
        [-0.2089, -0.2048,  0.0958,  ...,  0.3468, -0.2298,  0.2133]],
       device='cuda:0', grad_fn=<ToCopyBackward0>)

In [ ]:
sp.shape

torch.Size([10, 1024])

In [ ]:
all_embeds = model.transformer.wte.weight[:, :]

In [ ]:
all_embeds.to(device)

tensor([[-0.0115,  0.0031, -0.0073,  ..., -0.0526, -0.1757,  0.0257],
        [-0.0086,  0.0636, -0.0182,  ..., -0.0136, -0.1215,  0.0535],
        [ 0.0585,  0.0689,  0.0262,  ..., -0.1006, -0.1979, -0.0039],
        ...,
        [ 0.0016, -0.0441, -0.0517,  ..., -0.1008, -0.0087,  0.0264],
        [-0.1437, -0.0463, -0.0065,  ...,  0.0746, -0.0472, -0.0383],
        [ 0.0207, -0.0133, -0.0259,  ...,  0.0389, -0.0023,  0.0011]],
       device='cuda:0')

In [ ]:
token_list = []
for s in sp:
  min_idx = torch.norm(s.to(device) - all_embeds.unsqueeze(0), dim=1).argmin()
  token_list.append(min_idx.detach().item())
print(token_list)

[968, 805, 909, 671, 364, 671, 729, 433, 262, 729]


In [ ]:
tokens = tokenizer.convert_ids_to_tokens(token_list)
tokens

['ĠNew', 'man', 'Ġob', 'ade', 'ers', 'ade', 'ater', 'art', 'Ġthe', 'ater']

In [ ]:
tokenizer.convert_tokens_to_string(tokens)

' Newman obadeersadeaterart theater'